In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview")

: 

In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, DirectoryLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
import textwrap
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [1]:
from glob import glob
from utils import to_markdown
from urllib.parse import urlparse
from collections import defaultdict

In [4]:
def contains_course_info(url):
    url_prased = urlparse(url)
    return len(url_prased.path.split("/"))>5

Document(page_content='\n\n\n\r\n\tStevens Institute of Technology\xa0-\xa0Master of Science in Machine Learning\r\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCatalog Search\n\n\n\n\nSearch Options\n\n\nEntire Catalog\nPrograms\nCourses\n\nSearch\nhttp://stevens.smartcatalogiq.com\n4d3ddf27-e408-400c-b836-d1281514fc2f\n\n68aab440-d6be-464a-b585-3d808753e390\nprogram\n\n\n\n\n/Institutions/Stevens-Institution-of-Technology/json/2023-2024/Academic-Catalog-local.json/Institutions/Stevens-Institution-of-Technology/json/2023-2024/Academic-Catalog.json\n\r\n\t\tContents\r\n\t\n\n\n\nAboutOur History and MissionAccreditationAcademic IntegrityStudent LifeStudent ServicesUndergraduate EducationGraduate EducationTuition, Fees and Other Expenses for Undergraduate StudentsTuition, Fees and Other Expenses for Graduate StudentsFinancing a Stevens EducationSchool of BusinessSchool of Humanities, Arts and Social SciencesSchaefer School of Engineering and Scienc

In [5]:
files = []
with open("stevens_scraper/crawled_urls.txt") as fp:
    for line in fp:
        files.append(line.strip())
files = list(map(lambda x: x.lower(), files))
files = list(set(files))
files = list(filter(lambda file: "narrative-courses" not in file, files))
files = list(filter(lambda file: "javascript" not in file, files))
files = list(filter(lambda file: ".pdf" not in file, files))
files = list(filter(lambda file: ".aspx" not in file, files))
files = list(filter(lambda file: "degree-programs2" not in file, files))
files = list(filter(lambda file: "2023-2024" in file, files))
files = list(filter(lambda file: "#middle" not in file, files))
files = list(filter(contains_course_info , files))
print(len(files))

with open("stevens_scraper/crawled_urls_v1.txt", "w+") as fp:
    fp.write("\n".join(sorted(files, key=len)))

c:\Users\smend\miniconda3\envs\stevens\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
primary = defaultdict(list)
for url in files:
    url_prased = urlparse(url.lower())
    cat = url_prased.path.replace("/en/2023-2024/academic-catalog/", "").split("/")[0]
    primary[cat].append(url)

In [ ]:
school_department_keys = [
    "school-of-business",
    "department-of-chemical-engineering-and-materials-science",
    "department-of-electrical-and-computer-engineering",
    "department-of-mathematical-sciences",
    "department-of-chemistry-and-chemical-biology",
    "department-of-computer-science",
    "department-of-civil-environmental-and-ocean-engineering",
    "department-of-mechanical-engineering",
    "department-of-physics",
    "school-of-systems-and-enterprises",
    "schaefer-school-of-engineering-and-science",
    "department-of-biomedical-engineering",
    "school-of-humanities-arts-and-social-sciences"
]

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [7]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [8]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [   
        ("system", "You are an assistant for question-answering tasks related to Stevens Institute Of Technology."),
        ("human", """ 
        Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.
        Answer in markdown format and render tables without code 
        Question: {question}
        Context: {context}
        Answer:"""),
    ]
)
chat_template

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant for question-answering tasks related to Stevens Institute Of Technology.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=" \n        Use the following pieces of retrieved context to answer the question.\n        If you don't know the answer, just say that you don't know.\n        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.\n        Answer in markdown format and render tables without code \n        Question: {question}\n        Context: {context}\n        Answer:"))])

In [9]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# rag_chain = (
#     { "context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [12]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [13]:
response = rag_chain_with_source.invoke("What are the core courses  for MS in Machine Learning?")

In [14]:
response

{'context': [Document(page_content='Build statistical learning models and understand their strengths and limitations.\n\n\nProvide appropriate interpretation of classification or regression results.\n\n\n\xa0\n\n\r\n\t\t\tDegree Requirements\r\n\t\tThe program is a 30-credit degree program. Students are required to complete:\n\n\n4 machine learning core courses (12 credits)\n\n\n3 machine learning core electives\xa0 (9 credits)\xa0\n\n\n3 general electives (9 credits), which can be any graduate course\n\n\nStudents must maintain a minimum grade of a C or above in any course and a minimum GPA of 3.000.\r\n\t\t\tMachine Learning Core Requirements: Complete Four out of the Five Courses Mentioned Below:\r\n\t\t\n\nCS 541Artificial Intelligence3\n\nCS 559Machine Learning: Fundamentals and Applications3\n\nCS 560Statistical Machine Learning3\n\nCS 583Deep Learning3', metadata={'language': 'en', 'source': 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/department-of-computer

In [16]:
to_markdown(response["answer"])

> The core courses for an MS in Machine Learning are CS 541 Artificial Intelligence, CS 559 Machine Learning: Fundamentals and Applications, CS 560 Statistical Machine Learning, CS 583 Deep Learning, and students must complete four out of these five courses. Additionally, there is a selection of core electives and general electives, but the core electives and general electives are not strictly defined as core courses. The core electives include various specialized topics such as data mining, computer vision, health informatics, and others, while the general electives can be any graduate course.

In [24]:
# cleanup
vectorstore.delete_collection()